# Imports

In [1]:
import pydot
import graphviz
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

In [2]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, merge, concatenate, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import model_to_dot
from IPython.display import SVG
from keras.layers import dot
from tensorflow.keras.utils import plot_model

2021-09-01 11:27:46.373856: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-01 11:27:46.373994: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

# Load KNN  and predict

In [5]:
# imports

## dependencies

import joblib
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import pickle


import pydot
import graphviz
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, merge, concatenate, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import model_to_dot
from IPython.display import SVG
from keras.layers import dot
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

#!pip install scikit-learn==0.24.2

## files

# model_path = "../data/models_anime_map_knn_model.joblib"
# pickle.load(open('../model.sav', 'rb'))

def get_anime():
    anime_df_relevant_PG = pd.read_csv("../data/anime_df_relevant_PG.csv")
    return anime_df_relevant_PG.rename(columns={'MAL_ID' : 'anime_id'})

def get_data():
    data = pd.read_csv('../data/processed_data/active_users_df_10PlusRatings_partial.csv')
    return data

def get_model():
    return pickle.load(open('../model_knn.sav', 'rb'))

get_model()

def process_data():
    data_users_df = get_data()
    data_users_df['rating'] = data_users_df['rating']/10
    
    anime_df_relevant_PG = get_anime()
    anime_name_df = anime_df_relevant_PG[['anime_id','Name']]
    data_users_df_merge = data_users_df.merge(anime_name_df, on = 'anime_id', how='inner')
    pivot_df = data_users_df_merge.pivot_table(index='anime_id',columns='user_id',values='rating').fillna(0)
    
    anime_Genres_df = anime_df_relevant_PG[['anime_id','Genres']]
    anime_Genres_df_encoded = pd.concat(objs = [anime_Genres_df.drop(columns = 'Genres', axis =1), anime_Genres_df['Genres'].str.get_dummies(sep=", ")], axis = 1)
    anime_Genres_df_encoded = anime_Genres_df_encoded.set_index('anime_id')
    
    pivot_df = pivot_df.merge(anime_Genres_df_encoded, how='inner',left_index=True, right_index=True)
    anime_name_pivot_df = data_users_df_merge[['anime_id','Name']].drop_duplicates()
    anime_name_pivot_df = anime_name_pivot_df.sort_values('anime_id')
    anime_name_pivot_df = anime_name_pivot_df.reset_index().drop(columns = 'index')
    
    return pivot_df, anime_name_pivot_df

# predict

def recomendation_10PlusRatings(anime_name, nb_recomendation = 10):
    pivot_df, anime_name_pivot_df = process_data()
    model = get_model()
    index_nb = anime_name_pivot_df.index[anime_name_pivot_df['Name'] == anime_name].tolist()[0]
    distances, indices = model.kneighbors(pivot_df.iloc[index_nb,:].values.reshape(1, -1), n_neighbors = nb_recomendation + 1)
    return indices

# Test_df preparation before feeding to NeuMF

 - concat lists of recom
 - remove duplicates
 - add user_id
 - map anime name and its number to feed to model
 - put in model
 - predict probable user ratings for predicted anime from knn
 - sort by desc

In [6]:
#knn_recommend = pd.DataFrame(recomendation_10PlusRatings('Naruto'))

In [22]:
knn_recommend_id = recomendation_10PlusRatings('Naruto')

## Transform list of recom into dataset suitable for NeuMF, column of user number(single number, because user is one person) and anime names they like

In [8]:
# # load anime dataset to map anime name from recom to its anime_id to feed to neuMF
# def get_anime(): 
#     merged = pd.read_csv("../data/processed_data/merged_df_anime_users_relevant_PG.csv") 
#     merged.drop(columns=['Genres',"Studios"])
#     return merged
#     #return anime_df_relevant_PG.rename(columns={'MAL_ID' : 'anime_id'})

# merged = get_anime()

# merged.drop(columns=['Genres','Studios','user_id','rating'],inplace = True)

# merged.drop(columns=['user_id','rating'],inplace = True)

# merged.head()

# unique_names = merged.drop_duplicates('Name',keep='first')

# unique_names.shape

# #np.full(len(knn_recommend),fill_value = 0)
# knn recommendations names to anime_ids

# test_df_to_neuMF = 
# df3 = pd.DataFrame(test_to_neuMF, columns=['user_id', 'anime_id'])

#unique_names.head()

# knn_recommend

# int(unique_names[unique_names.Name=='Trigun']['anime_id'])



# #merged.anime_id.head()[merged["Name"].head() == 'Cowboy Bebop']

# recommend_anime_id = []
# for i,j in enumerate(knn_recommend):
#     id = int(unique_names[unique_names.Name==j]['anime_id'])
#     recommend_anime_id.append(id)

# recommend_anime_id

In [12]:
#DONT NEED
# Load anime_id full list
# anime_list = pd.read_csv("../data/processed_data/anime_df_relevant_PG.csv") 

# anime_list.rename(columns={'MAL_ID': 'anime_id'}, inplace=True)

# anime_list.columns

# anime_list.drop(columns=['Genres','Name','Studios'],inplace=True)

# anime_id_list = anime_list

## Create quasi user_id (perceptron requires it as input along with anime_id)

In [33]:
#column_user_id = np.zeros(len(knn_recommend_id[0]))

In [55]:
column_user_id = [20 for i in range(len(knn_recommend_id[0]))]

In [56]:
column_user_id

[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]

In [57]:
knn_recommend_id[0]

array([  10, 1496, 1326,  237, 5564, 3251, 1362, 4990,   11, 3726, 6964])

In [58]:
test = pd.DataFrame(list(knn_recommend_id[0]), columns =['anime_id'])

In [59]:
test['user_id'] = column_user_id

In [60]:
test

,anime_id,user_id
0,10,20
1,1496,20
2,1326,20
3,237,20
4,5564,20
5,3251,20
6,1362,20
7,4990,20
8,11,20
9,3726,20


In [66]:
def id_transform(dataset):
    anime_id_to_new_id = dict()
    id = 1
    for index, row in dataset.iterrows():
        if anime_id_to_new_id.get(row['anime_id']) is None:
            anime_id_to_new_id[row['anime_id']] = id
            dataset.at[index, 'anime_id'] = id
            id += 1
        else:
            dataset.at[index, 'anime_id'] = anime_id_to_new_id.get(row['anime_id'])
    user_id_to_new_id = dict()
    id = 1
    for index, row in dataset.iterrows():
        if user_id_to_new_id.get(row['user_id']) is None:
            user_id_to_new_id[row['user_id']] = id
            dataset.at[index, 'user_id'] = id
            id += 1
        else:
            dataset.at[index, 'user_id'] = user_id_to_new_id.get(row['user_id'])
    return dataset

In [68]:
test = id_transform(test)

# Load perceptron model

In [61]:
def load_model():
    model = tf.keras.models.load_model('../neuMFmodel.h5')
    #model = joblib.load('neuMFmodel.joblib')
    return model


In [62]:
model_neuMF = load_model()

In [70]:
def predict(test,model):
    y_pred = np.round(model.predict([test.user_id, test.anime_id]), decimals=2)
#     y_true = test.rating
#    mean_absolute_error(y_true, y_pred)
    return y_pred

In [71]:
predict(test,model_neuMF)

array([[7.46],
       [7.46],
       [7.46],
       [7.47],
       [7.46],
       [7.46],
       [7.46],
       [7.46],
       [7.46],
       [7.46],
       [7.46]], dtype=float32)